In [1]:
#from pyspark import SparkContext, SparkConf
#conf = SparkConf().setAppName("hi").setMaster("jj")
#sc = SparkContext(conf=conf)


Out[5]: [[2.0, 14.0, 33.0, 50.0]]

+---+------+------+------------------+
age|gender| name| occupation|
+---+------+------+------------------+
 19| Male| Mike|Computer Scientist|
 26| Male| Paul|Computer Scientist|
 25| Male| Bob|Computer Scientist|
 30|Female|Olivia| Accountant|
 32| Male| Rob|Computer Scientist|
 36|Female| Susan|Computer Scientist|
 35| Male| David| Accountant|
 44|Female| Emma| Accountant|
 32|Female| Lisa| Accountant|
+---+------+------+------------------+

+---+------+------+------------------+
age|gender| name| occupation|
+---+------+------+------------------+
 19| Male| Mike|Computer Scientist|
 26| Male| Paul|Computer Scientist|
 25| Male| Bob|Computer Scientist|
 30|Female|Olivia| Accountant|
 32| Male| Rob|Computer Scientist|
 36|Female| Susan|Computer Scientist|
 35| Male| David| Accountant|
 44|Female| Emma| Accountant|
 32|Female| Lisa| Accountant|
+---+------+------+------------------+

+----------+------------------+------+
count(age)| occupation|gender|
+----------+------------------+------+
 1| Accountant| Male|
 3|Computer Scientist| Male|
 1|Computer Scientist|Female|
 3| Accountant|Female|
+----------+------------------+------+

+------------------+-----+
 occupation|count|
+------------------+-----+
Computer Scientist| 5|
 Accountant| 4|
+------------------+-----+

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-3639460808179459> in <module> 
 1 y = df2 . groupby ( 'occupation' ) 
 ----> 2 y . show ( ) 

 AttributeError : 'GroupedData' object has no attribute 'show'

+------------------+------+-----+
 occupation|gender|count|
+------------------+------+-----+
 Accountant| Male| 1|
Computer Scientist| Male| 3|
Computer Scientist|Female| 1|
 Accountant|Female| 3|
+------------------+------+-----+

+------------------+-----+---+------+------+------------------+
 occupation|count|age|gender| name| occupation|
+------------------+-----+---+------+------+------------------+
Computer Scientist| 3| 26| Male| Paul|Computer Scientist|
Computer Scientist| 3| 25| Male| Bob|Computer Scientist|
 Accountant| 1| 30|Female|Olivia| Accountant|
Computer Scientist| 3| 32| Male| Rob|Computer Scientist|
Computer Scientist| 3| 36|Female| Susan|Computer Scientist|
 Accountant| 1| 35| Male| David| Accountant|
 Accountant| 1| 44|Female| Emma| Accountant|
 Accountant| 1| 32|Female| Lisa| Accountant|
+------------------+-----+---+------+------+------------------+

In [13]:
# getting number of unique movies in 1 million data 9742 movies
# getting number of unique users in 1 million data 610 users
#getting number of unique users in 27 million data 283228 users
#paths: /FileStore/tables/ratings.csv, /FileStore/tables/movies.csv,/FileStore/tables/ratings_1m.csv,/FileStore/tables/movies_1m.csv
import sys
#path_1 = sys.argv[1]
s_df = spark.read.format("csv").option("header", "true").load('/FileStore/tables/ratings_1m.csv')
a = [i.userId for i in s_df.select('userId').distinct().collect()]
len(a)

Out[2]: 610

In [14]:
import os
#reading the ratings file data and its header which is printed and shown below
#Make sure give the paths with respect to data
read_ratings_1m = sc.textFile("/FileStore/tables/ratings.csv")
read_ratings_1m_header = read_ratings_1m.take(1)[0]
read_ratings_1m_header

Out[3]: 'userId,movieId,rating,timestamp'

In [15]:
#Read the data in the foramt (userid,movieid,rating) list from csv such using filter and map transforamtion opration of spark
read_ratings_1m_data = read_ratings_1m.filter(lambda line : line != read_ratings_1m_header).map(lambda line:line.split(",")).map(lambda tokens:(int(tokens[0]),int(tokens[1]),float(tokens[2])))
print('the sample data look like in RDD')
read_ratings_1m_data.take(3)

the sample data look like in RDD
Out[4]: [(1, 307, 3.5), (1, 481, 3.5), (1, 1091, 1.5)]

In [16]:
# total ratings count
read_ratings_1m_data.count()

Out[5]: 27753444

In [17]:
#catching the data
read_ratings_1m_data.cache()

Out[6]: PythonRDD[1003] at RDD at PythonRDD.scala:59

In [18]:
##reading the movies file data and its header which is printed and shown below
read_movies_1m = sc.textFile("/FileStore/tables/movies.csv")
read_movies_1m_header = read_movies_1m.take(1)[0]
read_movies_1m_header
#read_movies_1m.take(3)

Out[7]: 'movieId,title,genres'

In [19]:
#Read the data in the foramt (userid,movieid,rating) list from csv such using filter and map transforamtion opration of spark
read_movies_1m_data = read_movies_1m.filter(lambda line : line != read_movies_1m_header).map(lambda line:line.split(",")).map(lambda tokens:(int(tokens[0]),tokens[1]))
read_movies_1m_data.cache()
print("movies RDD Look like Below")
read_movies_1m_data.take(3)

movies RDD Look like Below
Out[8]: [(1, 'Toy Story (1995)'),
 (2, 'Jumanji (1995)'),
 (3, 'Grumpier Old Men (1995)')]

In [20]:
#train_test_split the data of ratings
train, test = read_ratings_1m_data.randomSplit([7, 3], seed=0)
#train.take(3)
test.take(3)

Out[9]: [(1, 1091, 1.5), (1, 1590, 2.5), (1, 1591, 1.5)]

In [21]:
# hyperparameter tuning
from pyspark.mllib.recommendation import ALS
import math
def training_Als(rank_l):
  seed = 5
  iterations = 10
  reg_parameter = 0.1
  rank = rank_l
  error = []
  for i in rank:
    cf_model = ALS.train(train, i, seed = seed, iterations = iterations, lambda_=reg_parameter)
    test_predict = test.map(lambda tuple_element:(tuple_element[0],tuple_element[1]))
    prediction = cf_model.predictAll(test_predict).map(lambda r:((r[0], r[1]), r[2]))
    #joing this predicted ratings with original values for finding the accuracy of the model
    r_p = test.map(lambda x : ((int(x[0]), int(x[1])), float(x[2]))).join(prediction)
    error1 = math.sqrt(r_p.map(lambda r : (r[1][0] - r[1][1])**2).mean())
    error.append(error1)
    
  return error
  


In [22]:
# un comment this cell and run for visualizing the hyperparameter cell
# error 0.9066087316923477,0.9008785946214488,0.9005231357577965,0.9030239910882025,0.8980738472518457,0.8970183082152089,0.9010387404365356,0.8985533867210549
#rank = [5,6,7,8,9,10,11,12]
#error = training_Als(rank)

In [23]:
#plot showing hyperparamter rank i.e lantent features 
#import plotly.graph_objects as go
#from plotly.graph_objs import *
#fig = go.Figure()

#fig.add_trace(go.Scatter(
#    x= rank,
#    y=error,
#    mode="markers+text+lines",
#    text= error,
#    hovertext=rank,
#    marker=dict(
#      size=15,
#      color= "blue"
#     #set color equal to a variable
     
#E  )
#))


#fig.update_layout(title_text="Hover over the points to see the text")

#fig.show()

In [24]:
#traininf the model with best hyper parameter value rank. 
from pyspark.mllib.recommendation import ALS
import math

seed = 5
iterations = 10
reg_parameter = 0.1
rank = 8
error = 0

cf_model = ALS.train(train, rank, seed = seed, iterations = iterations, lambda_=reg_parameter)
test_predict = test.map(lambda tuple_element:(tuple_element[0],tuple_element[1]))
prediction = cf_model.predictAll(test_predict).map(lambda r:((r[0], r[1]), r[2]))
#joing this predicted ratings with original values for finding the accuracy of the model
r_p = test.map(lambda x : ((int(x[0]), int(x[1])), float(x[2]))).join(prediction)
error1 = math.sqrt(r_p.map(lambda r : (r[1][0] - r[1][1])**2).mean())

print("Root Mean Square Error")               
print(error1)
               

Root Mean Square Error
0.8162060462702274

In [25]:
# Normalizing the error
error1 = math.sqrt(r_p.map(lambda r : ((r[1][0] - r[1][1])**2)/25).mean())
error1

Out[14]: 0.16324120925404573

In [26]:
# counting movies ratings for Recommending to a user


ratings = read_ratings_1m_data.map(lambda x : (x[1],x[2])).groupByKey()
movie_avg_ratings = ratings.map(lambda x: (x[0], len(x[1]),sum([i for i in x[1]])/len(x[1])))
ratings.take(5)
movie_avg_ratings.take(5)

Out[15]: [(1296, 7470, 3.8941767068273094),
 (2352, 7517, 3.720167620061195),
 (828, 1736, 3.1474654377880182),
 (960, 173, 3.491329479768786),
 (2028, 54027, 4.05053029041035)]

In [27]:
# Adding new user to the datset
user_id = 0

# creading userid, movieid, ratings RDD for a new user (movie Ids are choosen randomly based on his interest)

new_u_ratings = [
     (0,260,4), # Star Wars (1977)
     (0,1,3), # Toy Story (1995)
     (0,16,3), # Casino (1995)
     (0,25,4), # Leaving Las Vegas (1995)
     (0,32,4), # Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
     (0,335,1), # Flintstones, The (1994)
     (0,379,1), # Timecop (1994)
     (0,296,3), # Pulp Fiction (1994)
     (0,858,5) , # Godfather, The (1972)
     (0,50,4) # Usual Suspects, The (1995)
    ]

new_u_ratings_RDD = sc.parallelize(new_u_ratings)
new_u_ratings_RDD.take(10)

# Adding this user data to the original user-ratings data

Added_new_ur_data =  read_ratings_1m_data.union(new_u_ratings_RDD)

In [28]:
# train the model again with added new user
clf_new_model = ALS.train(Added_new_ur_data, rank, seed = seed, iterations = iterations, lambda_=reg_parameter)


In [29]:
# getting new user ids
new_ur_movie_ids = map(lambda x: x[1], new_u_ratings)
new_ur_movie_ids

Out[18]: <map at 0x7fe69403e898>

In [30]:
#Getting recommnedation for new user un rated movies
new_user_unrated_movies_RDD = read_movies_1m_data.filter(lambda x: x[0] not in new_ur_movie_ids).map(lambda x: (user_id, x[0]))
new_user_recommendations_RDD = clf_new_model.predictAll(new_user_unrated_movies_RDD)


In [31]:
# Showing new user input to the model
new_user_recommendations_RDD.take(3)
new_user_unrated_movies_RDD.take(5)

Out[20]: [(0, 2), (0, 3), (0, 4), (0, 5), (0, 6)]

In [32]:
# predicitnng new user ratings
pred_ratings = new_user_recommendations_RDD.map(lambda x : (x[1],x[2])).sortByKey()
pred_ratings.take(4)

Out[21]: [(2, 1.9381774091699322),
 (3, 1.9328798489940882),
 (4, 1.2678377614282974),
 (5, 1.545261021858249)]

In [33]:
# combining predicted values with titles
predicted_r_movie_titles = pred_ratings.join(read_movies_1m_data).join(movie_avg_ratings)
predicted_r_movie_titles.take(3)

Out[22]: [(24, ((1.90549907886275, 'Powder (1995)'), 9191)),
 (48, ((1.6958876095011368, 'Pocahontas (1995)'), 16358)),
 (72, ((2.7393221685779903, 'Kicking and Screaming (1995)'), 1334))]

In [34]:
# falttinng the predicted values
flating_predictions = predicted_r_movie_titles.map(lambda x : (x[0],x[1][0][1],x[1][0][0],x[1][1]))
result = flating_predictions.filter(lambda x : x[3] > 25).takeOrdered(25, key=lambda x: -x[2])

In [35]:
# displaying the recpredited recommendations
flating_predictions.take(3)

Out[24]: [(24, 'Powder (1995)', 1.90549907886275, 9191),
 (48, 'Pocahontas (1995)', 1.6958876095011368, 16358),
 (72, 'Kicking and Screaming (1995)', 2.7393221685779903, 1334)]

In [36]:
# Displaying the results
result

Out[25]: [(109370, 'Magic & Bird: A Courtship of Rivals (2010)', 4.205458047054151, 27),
 (4171, "Long Night's Journey Into Day (2000)", 4.034671592120159, 35),
 (858, '"Godfather', 3.9401285962269315, 60904),
 (139086,
 'Small Potatoes - Who Killed the USFL? (2009)',
 3.9250156017837905,
 26),
 (171495, 'Cosmos', 3.9183881373494582, 157),
 (79677, '"Two Escobars', 3.9133231390548304, 78),
 (1221, '"Godfather: Part II', 3.8746254985856208, 38875),
 (173351, 'Wow! A Talking Fish! (1983)', 3.8728742513819707, 47),
 (108043, 'Milius (2013)', 3.8611850190449717, 44),
 (160317, 'Roots (2016)', 3.860586669993372, 28),
 (116002, 'History of the Eagles (2013)', 3.8520613377611097, 32),
 (91762, '"Last Lions', 3.8439770709231045, 38),
 (70186,
 'Heimat - A Chronicle of Germany (Heimat - Eine deutsche Chronik) (1984)',
 3.8351099378886944,
 35),
 (296, 'Pulp Fiction (1994)', 3.801915117453738, 92406),
 (100553, 'Frozen Planet (2011)', 3.793612933978338, 402),
 (182615, 'The Landlord (2007)', 3.7694492637625405, 26),
 (165239, 'Supersonic (2016)', 3.7613698898885604, 51),
 (128981,
 'Music for One Apartment and Six Drummers (2001)',
 3.758114602698226,
 31),
 (69483, '"Not on Your Life (Verdugo', 3.7437483443350423, 28),
 (104069, 'Louis C.K.: Oh My God (2013)', 3.7311551931029308, 918),
 (136447,
 'George Carlin: You Are All Diseased (1999)',
 3.7299635638346347,
 163),
 (95973, '"Classe américaine', 3.727265394410656, 38),
 (171011, 'Planet Earth II (2016)', 3.7197041565270705, 853),
 (170705, 'Band of Brothers (2001)', 3.719112688606158, 984),
 (318, '"Shawshank Redemption', 3.716193365655629, 97999)]